In [327]:
import re
import subprocess
from pathlib import Path

import dicom2nifti
import pydicom
import shutil
import tempfile
import nilearn.image
import numpy as npy
import pandas as pd
import nibabel as nib
from p_tqdm import p_map
from hashlib import sha3_256

In [328]:
# parameters
project = 'TUE1014TSHOD'
in_dir = Path(f'/media/nora/import_move/')/project
out_dir = Path(f'/media/dataheppt1/raheppt1/dcm_out')/project
out_dir.mkdir(exist_ok=True)
ano_dir = out_dir/'anonymization'/project
ano_dir.mkdir(exist_ok=True, parents=True)
nora_dir = Path('/media/nora/imgdata')/project
num_cpus = 7

### DICOM to nifti

#### Hashing

In [329]:
def name_to_hash(patient, study):
    hash_name_long = sha3_256(patient.encode("utf-8")).hexdigest()
    hash_seq_long = sha3_256(study.encode("utf-8")).hexdigest()
    hash_name = hash_name_long[:7]
    seq_name = hash_seq_long[:3]
    complete_hash = hash_name + seq_name
    return complete_hash

#### PET SUV conversion

In [330]:
def conv_time(time_str):
    return (float(time_str[:2]) * 3600 + float(time_str[2:4]) * 60 + float(time_str[4:13]))

def calculate_suv_factor(dcm_path):
    ds = pydicom.dcmread(str(dcm_path))
    total_dose = ds.RadiopharmaceuticalInformationSequence[0].RadionuclideTotalDose
    start_time = ds.RadiopharmaceuticalInformationSequence[0].RadiopharmaceuticalStartTime
    half_life = ds.RadiopharmaceuticalInformationSequence[0].RadionuclideHalfLife
    acq_time = ds.AcquisitionTime
    weight = ds.PatientWeight
    time_diff = conv_time(acq_time) - conv_time(start_time)
    act_dose = total_dose * 0.5 ** (time_diff / half_life)
    suv_factor = 1000 * weight / act_dose
    return suv_factor

def convert_pet(pet, suv_factor=1.0):
    affine = pet.affine
    pet_data = pet.get_fdata()
    pet_suv_data = (pet_data*suv_factor).astype(np.float32)
    pet_suv = nib.Nifti1Image(pet_suv_data, affine)
    return pet_suv 

#### Dicom Preprocessing

In [335]:
def process_directory(dcm_dir):
    info = {}

    # parsing
    pet_dcm_header = next(dcm_dir.glob('**/P*.dcm'))
    ct_dcm_header = next(dcm_dir.glob('**/C*.dcm'))
    ds = pydicom.dcmread(str(pet_dcm_header))

    # create directories
    subject_hash = name_to_hash(ds.PatientName, ds.StudyID)
    subject_dir = out_dir/subject_hash
    subject_dir.mkdir(exist_ok=True)
    shutil.copy(pet_dcm_header, ano_dir/f'{subject_hash}_pet.dcm')
    shutil.copy(ct_dcm_header, ano_dir/f'{subject_hash}_ct.dcm')
    temp = tempfile.TemporaryDirectory()
    temp_path = Path(temp.name)

    # get subject info
    suv_factor = calculate_suv_factor(pet_dcm_header)
    info['Project'] = [project]
    info['Hash'] = [subject_hash]
    info['Name'] = [str(ds.PatientName)]
    info['PatientID'] = [str(ds.PatientID)]
    info['StudyID']= [str(ds.StudyID)]
    info['BirthDate'] = [str(ds.PatientBirthDate)]
    info['StudyDate'] = [str(ds.StudyDate)]
    info['AccessionNumber'] = [str(ds.AccessionNumber)]
    info['Age'] = [int(ds.PatientAge[:-1])]
    info['SUVfactor'] = [suv_factor]

    # convert dicom to nifti
    dicom2nifti.convert_directory(dcm_dir, temp_path, compression=True, reorient=True)

    # store niftis
    ct_path = next(temp_path.glob('*gk*.nii.gz'))
    pet_path = next(temp_path.glob('*pet*.nii.gz'))
    pet = nib.load(pet_path)
    ct = nib.load(ct_path)

    pet_suv = convert_pet(pet, suv_factor)
    nib.save(pet, subject_dir/f'{subject_hash}_petcorr.nii.gz')
    nib.save(pet_suv, subject_dir/f'{subject_hash}_petsuv.nii.gz')
    nib.save(ct, subject_dir/f'{subject_hash}_ct.nii.gz')

    if nora_dir:
        name = info['Name'][0].split('^')
        nora_subject_dir = nora_dir/f'{name[0]}_{name[1]}_{info["PatientID"][0]}/{info["StudyID"][0]}_{info["StudyDate"][0]}'
        nora_subject_dir.mkdir(exist_ok=True, parents=True)
        nib.save(pet_suv, nora_subject_dir/f'{subject_hash}_petsuv.nii.gz')
        nib.save(ct, nora_subject_dir/f'{subject_hash}_ct.nii.gz')
        subprocess.run(['nora', '-p', project, '--add', str(nora_subject_dir)])

    temp.cleanup()
    return info

In [336]:
# multiprocessing
res = p_map(process_directory, list(in_dir.glob('*')), num_cpus=num_cpus)

In [ ]:
df = pd.concat([pd.DataFrame.from_dict(x) for x in res])
df.to_csv(ano_dir.parent/f'{project}.csv')